# **Re-gridding ship radiosonde netcdf files**
### [*re-gridded pressure resolution to match land launched radiosondes*]
#### *created by Selina Bolella and Alex Kinsella via '24 EKAMSAT campaign*

In [32]:
from affine import Affine
import geopandas as gpd
import numpy as np
from rasterio import features
import xarray as xr
import glob
import os
import metpy

In [34]:
# EKAMSAT 2024 Radiosondes for LEG 1

# Call path of .nc files of daily sonde data
paths = glob.glob(os.path.join("/Volumes/cruiseshare/For_Science/Notre Dame Data/Leg1/Radiosondes/netcdf/*_HSPOTINT.nc"))

# Loop daily data variables
for f in range(len(paths)):
    data = xr.open_dataset(paths[f])
    temp = data['T']
    pres = data['P']
    hght = data['height'] # LEG 1
    td = data['Td'] # LEG 1
    
    # Don't have u and v... need to calculate it!
    wd = data["wd"]
    ws = data["ws"]
    
    # Convert direction from degrees to radians
    direction_rad = np.deg2rad(wd)

    # Calculate u and v components
    u = -ws * np.sin(direction_rad)
    v = -ws * np.cos(direction_rad)

    ## Calculate es & mr to get q:
    
    # Calculate saturation vapor pressure
    es = 6.112 * np.exp((17.67*td)/(td+243.5))
    
    # Calculate Mixing Ratio
    mr = ((0.622 * es)/(pres-es))
    
    # Calculate SPECIFIC HUMIDITY (q)
    q = mr/(1+mr)

    # Set resolution for re-gridding
    dp = 5; #Resolution
    p = np.arange(1020, -5, -dp) #Set levels

    # Create lists for re-gridded variables
    temp_g = []
    hght_g = []
    q_g = []
    u_g = []
    v_g = []

    # Gridded every 5 mb
    for i in range(len(p)):
        temp_g.append((temp.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        hght_g.append((hght.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        q_g.append((q.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        u_g.append((u.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        v_g.append((v.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        #print(c.mean(skipna=True).values)

    # Assign each new gridded variable into a DataArray
    temp_grid = xr.DataArray(
        temp_g, 
        dims = "p",
        coords = {"p" : p},
        name = "temp"
    )

    hght_grid = xr.DataArray(
        hght_g, 
        dims = "p",
        coords = {"p" : p},
        name = "hght"
    )

    q_grid = xr.DataArray(
        q_g, 
        dims = "p",
        coords = {"p" : p},
        name = "q"
    )

    u_grid = xr.DataArray(
        u_g, 
        dims = "p",
        coords = {"p" : p},
        name = "u"
    )

    v_grid = xr.DataArray(
        v_g, 
        dims = "p",
        coords = {"p" : p},
        name = "v"
    )

    # Now save all new regridded variables into a dataset
    ds = xr.Dataset({"temp" : temp_grid, "hght" : hght_grid, "q" : q_grid, "u" : u_grid, "v" : v_grid})
    
    # Save to netcdf... paths[80:92]-> just means the grid date on path example: (202405230000)
    ds.to_netcdf("/Volumes/cruiseshare/For_Science/Situational_Awareness_Processing/data/radiosonde/grid_sondes/TGT_"+paths[f][80:92]+"_gridded.nc")

In [35]:
# EKAMSAT 2024 Radiosondes for LEG 2 

# Call path of .nc files of daily sonde data
paths = glob.glob(os.path.join("/Volumes/cruiseshare/For_Science/Notre Dame Data/Leg2/Radiosondes/netcdf/*_HSPOTINT.nc"))

# Loop daily data variables
for f in range(len(paths)):
    data = xr.open_dataset(paths[f])
    temp = data['T']
    pres = data['P']
    hght = data['gp_height']
    q = data['q']
    u = data['u']
    v = data['v']
    
    # Set resolution for re-gridding
    dp = 5; #Resolution
    p = np.arange(1020, -5, -dp) #Set levels

    
    # Create lists for re-gridded variables
    temp_g = []
    hght_g = []
    q_g = []
    u_g = []
    v_g = []

    # Gridded every 5 mb
    for i in range(len(p)):
        temp_g.append((temp.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        hght_g.append((hght.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        q_g.append((q.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        u_g.append((u.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        v_g.append((v.where((pres>p[i] - dp/2) & (pres<p[i] + dp/2))).mean(skipna=True).values)
        #print(c.mean(skipna=True).values)

    # Assign each new gridded variable into a DataArray
    temp_grid = xr.DataArray(
        temp_g, 
        dims = "p",
        coords = {"p" : p},
        name = "temp"
    )

    hght_grid = xr.DataArray(
        hght_g, 
        dims = "p",
        coords = {"p" : p},
        name = "hght"
    )

    q_grid = xr.DataArray(
        q_g, 
        dims = "p",
        coords = {"p" : p},
        name = "q"
    )

    u_grid = xr.DataArray(
        u_g, 
        dims = "p",
        coords = {"p" : p},
        name = "u"
    )

    v_grid = xr.DataArray(
        v_g, 
        dims = "p",
        coords = {"p" : p},
        name = "v"
    )
    
    # Now save all new regridded variables into a dataset
    ds = xr.Dataset({"temp" : temp_grid, "hght" : hght_grid, "q" : q_grid, "u" : u_grid, "v" : v_grid})
    
    # Save to netcdf... paths[80:92]-> just means the grid date on path example: (202405230000)
    ds.to_netcdf("/Volumes/cruiseshare/For_Science/Situational_Awareness_Processing/data/radiosonde/grid_sondes/TGT_"+paths[f][80:92]+"_gridded.nc")

In [23]:
paths = glob.glob(os.path.join("/Volumes/cruiseshare/For_Science/Notre Dame Data/Leg2/Radiosondes/netcdf/*_HSPOTINT.nc"))


In [28]:
paths[4][80:92]

'202405261200'